## python function for extracting a window of data from a tiff file

In [4]:
from osgeo import gdal,gdal_array                                                                                                         
import osr                                                                                                                                
from bunch import bunchify      

This expands the pmap.py module (use from pmap import getmap) 

In [5]:
def getmap(filename,buf=None):                                                                                                                     
        dic={}                                                                                                                               
        dataset=gdal.Open(filename)                                                                                                          
        dic['DRIVER']= dataset.GetDriver().LongName                                                                                          
        dic['NCOLS']=dataset.RasterXSize                                                                                                     
        dic['NROWS']=dataset.RasterYSize                                                                                                     
        dic['GeoTr']=dataset.GetGeoTransform()                                                                                               
        dic['Proj']=dataset.GetProjection()                                                                                                  
        geotransform = dataset.GetGeoTransform()                                                                                             
        if not geotransform is None:                                                                                                         
            dic['XSTART']=geotransform[0]                                                                                                     
            dic['YSTART']=geotransform[3]                                                                                                     
            dic['CELLSIZE']=geotransform[1]                                                                                                   
                                                                                                                                          
        v = dataset.GetRasterBand(1)
        if buf:
            [minlon,maxlon,minlat,maxlat]=buf
            
            gt=dic['GeoTr']
            minx = gt[0]                                                                                                                            
            miny = gt[3] + dic['NCOLS']*gt[4] + dic['NROWS']*gt[5]                                                                                               
            maxx = gt[0] + dic['NCOLS']*gt[1] + dic['NROWS']*gt[2]                                                                                               
            maxy = gt[3]                                                                                                                            
                                                                                                                                          
            lon=np.linspace(minx,maxx,dic['NCOLS'],endpoint=True)                                                                                          
            lat=np.linspace(miny,maxy,dic['NROWS'],endpoint=True)                                                                                         

            lat=lat[::-1]
                                                                                                                                          
            i1=np.abs(lon-minlon).argmin()                                                                                                        
            if lon[i1] > minlon: i1=i1-1                                                                                                          
            i2=np.abs(lon-maxlon).argmin()                                                                                                        
            if lon[i2] < maxlon: i2=i2+1                                                                                                          
                                                                                                                                          
            j1=np.abs(lat-minlat).argmin()                                                                                                        
            if lat[j1] > minlat: j1=j1-1                                                                                                          
            j2=np.abs(lat-maxlat).argmin()                                                                                                        
            if lat[j2] < maxlat: j2=j2+1                                                                                                          
                
            lons, lats = np.meshgrid(lon[i1:i2],lat[j2:j1])
            dic['lons'] = lons
            dic['lats'] = lats
            dic['data'] = v.ReadAsArray(i1,j2,i2-i1,j1-j2)
            
            #update the geo reference
            tl = list(dic['GeoTr'])
            tl[0] = lons.min()
            tl[-3] = lats.max()
            dic['GeoTr'] = tuple(tl)

            dic['NCOLS']=lons.shape[1]
            dic['NROWS']=lats.shape[0]
        else:
           
            dic['lons'] = []
            dic['lats'] = []
            dic['data']=v.ReadAsArray()                                                                                       
        
        dic['nan']=v.GetNoDataValue()                                                                                                        
                                                                                                                                          
        dataset=None                                                                                                                         
                                                                                                                                          
        return bunchify(dic)                                                                                                                 


In [6]:
filename='tmp/wind10mmalakas18w2016091600.tif'

In [8]:
data = getmap(filename)

In [9]:
data

Bunch(CELLSIZE=0.04999999999999716, DRIVER='PCRaster Raster File', GeoTr=(106.75, 0.04999999999999716, 0.0, 53.1, 0.0, -0.04999999999999716), NCOLS=1225, NROWS=942, Proj='', XSTART=106.75, YSTART=53.1, data=array([[-999., -999., -999., ..., -999., -999., -999.],
       [-999., -999., -999., ..., -999., -999., -999.],
       [-999., -999., -999., ..., -999., -999., -999.],
       ..., 
       [-999., -999., -999., ..., -999., -999., -999.],
       [-999., -999., -999., ..., -999., -999., -999.],
       [-999., -999., -999., ..., -999., -999., -999.]], dtype=float32), lats=[], lons=[], nan=-3.4028234663852886e+38)

## retrieve a window 

In [7]:
data = getmap(filename,buf=[minlon,maxlon,minlat,maxlat])

NameError: name 'minlon' is not defined